In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import openpyxl
from rdkit.Chem.Draw import IPythonConsole, MolsToGridImage

#Show mols in dataframes
from rdkit.Chem import PandasTools
from rdkit import Chem
from rdkit.Chem.Draw import MolsToGridImage
from IPython.core.display import HTML
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
from rdkit.Chem import rdDepictor
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
import os


In [5]:
TOP = os.getcwd().replace('notebooks', '')
raw_dir = TOP + 'data/raw/'
interim_dir = TOP + 'data/interim/'
external_dir = TOP + 'data/external/'
figures_dir = TOP + 'reports/figures/'
processed_dir = TOP + 'data/processed/'

In [3]:
import pickle

In [9]:
pwd

'/home/grace/Documents/python/aim/notebooks'

In [11]:
comparison = pickle.load(open(processed_dir+'comparison.pkl', 'rb' ))

AttributeError: Can't get attribute 'new_block' on <module 'pandas.core.internals.blocks' from '/home/grace/anaconda3/envs/pfas_dap/lib/python3.8/site-packages/pandas/core/internals/blocks.py'>

In [12]:
comparison_df = pd.read_csv(processed_dir+'comparison_matrix.csv')

In [23]:
comparison_df.iloc[0,1]

'[[    0   274]\n [16570  9231]]'

#### Read in the FP files from AIM and ToxPrints

In [28]:
aim = pd.read_csv(processed_dir+'AIM_output.txt', sep = ';')

In [44]:
aim[aim['M_CORINA_SYMPHONY_ERRORS_[STRING]'] == 'No errors']

,M_NAME,CH3 [aliphatic carbon],CH2 [aliphatic carbon],CH [aliphatic carbon],"C [aliphatic carbon No H, not tert]","C [aliphatic carbon No H, not tert].1","C [aliphatic carbon No H, not tert].2",=CH2 [olefinic carbon],=CH or =C [olefinc carbon],#C [acetylenic carbon],...,=C (C#N) C#N [dicyano] correction,Aromatic nitrogen C C(=S) N struct corr.,Cyclic ketoxime(>C=N OH) [1 olefin attach],C=N O sulfur(aliphatic) [iminoxy] correc,S N( C(=O) C) C(=O) C struct correction,Sulfur(+4) charged halide type,Ring rx: S=P(O)(O)O / 5 mem (2 nitrogen),O=P( O)( O) C OH correction,M_COMPOUND_HISTORY_[STRING],M_CORINA_SYMPHONY_ERRORS_[STRING]
0,DTXSID1045950,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Original compound,No errors
1,DTXSID1022762,1,1,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,Original compound,No errors
2,DTXSID8048789,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Original compound,No errors
3,DTXSID1045164,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Original compound,No errors
4,DTXSID9020247,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Original compound,No errors
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26069,DTXSID3024158,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Original compound,No errors
26071,DTXSID2023981,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Original compound,No errors
26072,DTXSID8023975,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Original compound,No errors
26073,DTXSID5024217,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Original compound,No errors


In [30]:
txps = pd.read_csv(processed_dir+'toxprintoutput.txt', sep = ';')

In [31]:
aim.head()

,M_NAME,CH3 [aliphatic carbon],CH2 [aliphatic carbon],CH [aliphatic carbon],"C [aliphatic carbon No H, not tert]","C [aliphatic carbon No H, not tert].1","C [aliphatic carbon No H, not tert].2",=CH2 [olefinic carbon],=CH or =C [olefinc carbon],#C [acetylenic carbon],...,=C (C#N) C#N [dicyano] correction,Aromatic nitrogen C C(=S) N struct corr.,Cyclic ketoxime(>C=N OH) [1 olefin attach],C=N O sulfur(aliphatic) [iminoxy] correc,S N( C(=O) C) C(=O) C struct correction,Sulfur(+4) charged halide type,Ring rx: S=P(O)(O)O / 5 mem (2 nitrogen),O=P( O)( O) C OH correction,M_COMPOUND_HISTORY_[STRING],M_CORINA_SYMPHONY_ERRORS_[STRING]
0,DTXSID1045950,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Original compound,No errors
1,DTXSID1022762,1,1,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,Original compound,No errors
2,DTXSID8048789,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Original compound,No errors
3,DTXSID1045164,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Original compound,No errors
4,DTXSID9020247,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Original compound,No errors


In [46]:
aim = aim.iloc[:,:-2]

In [48]:
aim = aim.set_index('M_NAME')

In [49]:
aim.head()

,CH3 [aliphatic carbon],CH2 [aliphatic carbon],CH [aliphatic carbon],"C [aliphatic carbon No H, not tert]","C [aliphatic carbon No H, not tert].1","C [aliphatic carbon No H, not tert].2",=CH2 [olefinic carbon],=CH or =C [olefinc carbon],#C [acetylenic carbon],"OH [hydroxy, aliphatic attach]",...,N CH=N OH [oxime type] correction,Ring rx: OH ortho to C=N N carbon,=C (C#N) C#N [dicyano] correction,Aromatic nitrogen C C(=S) N struct corr.,Cyclic ketoxime(>C=N OH) [1 olefin attach],C=N O sulfur(aliphatic) [iminoxy] correc,S N( C(=O) C) C(=O) C struct correction,Sulfur(+4) charged halide type,Ring rx: S=P(O)(O)O / 5 mem (2 nitrogen),O=P( O)( O) C OH correction
M_NAME,,,,,,,,,,,,,,,,,,,,,
DTXSID1045950,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DTXSID1022762,1,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
DTXSID8048789,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DTXSID1045164,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DTXSID9020247,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
txps = txps[txps['M_CORINA_SYMPHONY_ERRORS_[STRING]'] == 'No errors']

In [39]:
txps = txps.iloc[:,:-2]

In [41]:
txps = txps.set_index('M_NAME')

In [42]:
txps

,atom:element_main_group,atom:element_metal_group_I_II,atom:element_metal_group_III,atom:element_metal_metalloid,atom:element_metal_poor_metal,atom:element_metal_transistion_metal,atom:element_noble_gas,bond:C#N_cyano_acylcyanide,bond:C#N_cyano_cyanamide,bond:C#N_cyano_cyanohydrin,...,ring:polycycle_bicyclo_propene,ring:polycycle_spiro_[2.2]pentane,ring:polycycle_spiro_[2.5]octane,ring:polycycle_spiro_[4.5]decane,ring:polycycle_spiro_1_4-dioxaspiro[4.5]decane,ring:polycycle_tricyclo_[3.5.5]_cyclopropa[cd]pentalene,ring:polycycle_tricyclo_[3.7.7]bullvalene,ring:polycycle_tricyclo_[3.7.7]semibullvalene,ring:polycycle_tricyclo_adamantane,ring:polycycle_tricyclo_benzvalene
M_NAME,,,,,,,,,,,,,,,,,,,,,
DTXSID1045950,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DTXSID1022762,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DTXSID8048789,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DTXSID1045164,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DTXSID9020247,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DTXSID3024158,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DTXSID2023981,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DTXSID8023975,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Computing enrichments on a subset of records
- Looking for an alternative means of creating that crosstab matrix of ToxPrints vs AIM fingerprints but storing that in long matrix and computing Fisher statistics to more readily identify the interesting cases..
- Still a WIP

In [74]:
aim_mini = aim.iloc[:500,:50]
txps_mini = txps.iloc[:500, :50]

In [66]:
import scipy.stats as stats

In [121]:
def create_enrich(FP, Assay):
    res = []
    for e in Assay.columns:
        for i, column in FP.iteritems():
            #print(i,column)
            if pd.crosstab(Assay[e].values,column).shape == (2,2):
                a = pd.crosstab(Assay[e].values,column)
                #print(a)
                tn, fp, fn, tp = a.iloc[0,0], a.iloc[0,1], a.iloc[1,0], a.iloc[1,1]
                tested_active = (tp + fp)
                all_tested = tp+fn+tn+fp
                oddsratio, pvalue = stats.fisher_exact(a,alternative='greater')
                res.append([e, i, oddsratio, pvalue, tn, fp, fn, tp, all_tested])
    return res

In [122]:
res = create_enrich(aim_mini, txps_mini)

In [123]:
def convert(lst):
    df = pd.DataFrame(lst)
    df = df.rename(columns = {0: 'TxP', 1: 'AIM', 2: 'Odds Ratio', 3 : 'P-Value', 4: 'TN', 5: 'FP', 6: 'FN', 7: 'TP',  8: 'All Tested'})
    df = df.sort_values(by=['Odds Ratio','P-Value'], ascending = [True, False])
    return df

In [124]:
res_df = convert(res)

In [125]:
res_df

,TxP,AIM,Odds Ratio,P-Value,TN,FP,FN,TP,All Tested
5,atom:element_metal_group_I_II,=CH2 [olefinic carbon],0.0,1.000000,459,14,27,0,500
7,atom:element_metal_group_I_II,#C [acetylenic carbon],0.0,1.000000,471,2,27,0,500
10,atom:element_metal_group_I_II,"O [oxygen, aliphatic attach].1",0.0,1.000000,471,2,27,0,500
11,atom:element_metal_group_I_II,"O [oxygen, aliphatic attach].2",0.0,1.000000,469,4,27,0,500
16,atom:element_metal_group_I_II,"CL [chlorine, aliphatic attach].1",0.0,1.000000,471,2,27,0,500
...,...,...,...,...,...,...,...,...,...
441,bond:C(~Z)~C~Q_b-halocarbonyl,"COOH [acid, aliphatic attach]",inf,0.112000,444,55,0,1,500
472,bond:C(~Z)~C~Q_haloether_pyrimidine_2-halo-,"CL [chlorine, aromatic attach]",inf,0.104000,448,51,0,1,500
99,atom:element_metal_poor_metal,"O [oxygen, aliphatic attach]",inf,0.102000,449,50,0,1,500
454,bond:C(~Z)~C~Q_haloether_pyrimidine_2-halo-,"C [aliphatic carbon No H, not tert].1",inf,0.050000,475,24,0,1,500


In [120]:
b = pd.crosstab(aim_mini[' CH3 [aliphatic carbon]'], txps_mini['bond:C(~Z)~C~Q_a-halocarbonyl'])
tn, fp, fn, tp = b.iloc[0,0], b.iloc[0,1], b.iloc[1,0], b.iloc[1,1]
print(b)
print(tn, fp, fn, tp)

bond:C(~Z)~C~Q_a-halocarbonyl    0  1
 CH3 [aliphatic carbon]              
0                              167  4
1                              325  2
167 4 325 2


In [126]:
res_df[res_df['AIM'] ==  ' CH3 [aliphatic carbon]']

,TxP,AIM,Odds Ratio,P-Value,TN,FP,FN,TP,All Tested
180,bond:C#N_cyano_cyanamide,CH3 [aliphatic carbon],0.000000,1.000000,172,327,1,0,500
405,bond:C(~Z)~C~Q_b-halocarbonyl,CH3 [aliphatic carbon],0.000000,1.000000,172,327,1,0,500
450,bond:C(~Z)~C~Q_haloether_pyrimidine_2-halo-,CH3 [aliphatic carbon],0.000000,1.000000,172,327,1,0,500
495,bond:C(=O)N_carbamate_dithio,CH3 [aliphatic carbon],0.000000,1.000000,172,327,1,0,500
135,atom:element_metal_transistion_metal,CH3 [aliphatic carbon],0.129601,0.995227,169,326,4,1,500
315,bond:C(~Z)~C~Q_a-halocarbonyl,CH3 [aliphatic carbon],0.260000,0.979463,169,325,4,2,500
360,bond:C(~Z)~C~Q_a-haloether,CH3 [aliphatic carbon],0.262270,0.959048,171,326,2,1,500
270,bond:C#N_nitrile,CH3 [aliphatic carbon],0.311111,0.977057,168,324,5,3,500
225,bond:C#N_nitrile_generic,CH3 [aliphatic carbon],0.344685,0.976790,167,323,6,4,500
1170,bond:C(=O)O_carboxylicEster_alkenyl,CH3 [aliphatic carbon],0.398611,0.980670,164,320,9,7,500


In [127]:
def final_df(df):
    return df[(df['Odds Ratio'] >= 3) & (df['P-Value'] < 0.05 ) & (df['TP'] >=3)]


In [128]:
final_df(res_df)

,TxP,AIM,Odds Ratio,P-Value,TN,FP,FN,TP,All Tested
25,atom:element_metal_group_I_II,"N [aliphatic N, one aromatic attach]",3.223207,0.006983,400,73,17,10,500
981,bond:C(=O)O_carboxylicAcid_alkyl,"COOH [acid, aliphatic attach]",3.325581,0.001555,407,43,37,13,500
233,bond:C#N_nitrile_generic,"OH [hydroxy, aliphatic attach]",4.903614,0.018317,407,83,5,5,500
1247,bond:C(=O)O_carboxylicEster_alkyl,"C#N [cyano, aliphatic attach]",5.424490,0.040829,443,5,49,3,500
1296,bond:C(=O)O_carboxylicEster_aromatic,"COOH [acid, aliphatic attach]",5.615385,0.017779,438,52,6,4,500
935,bond:C(=O)O_carboxylicAcid_alkenyl,"NO2 [nitro, aromatic attach]",5.647773,0.028561,465,19,13,3,500
1054,bond:C(=O)O_carboxylicAcid_generic,"Br [bromine, aliphatic attach]",5.767123,0.047959,421,3,73,3,500
255,bond:C#N_nitrile_generic,"C(=O) [carbonyl, aliphatic attach]",8.321429,0.013102,466,24,7,3,500
611,bond:C(=O)N_carbamate,"O [oxygen, one aromatic attach]",8.800000,0.002215,440,50,5,5,500
964,bond:C(=O)O_carboxylicAcid_alkyl,"Br [bromine, aliphatic attach]",9.510638,0.015193,447,3,47,3,500


### Comparing with Matt's code and also to look at the array stored in the cell.

In [109]:
new_df = pd.DataFrame(columns = aim_mini.columns, index = txps_mini.columns)

In [111]:
for i, col in enumerate(aim_mini):
    a = []
    for j, col in enumerate(txps_mini):
        df_cross = pd.crosstab(aim_mini.iloc[:,i],txps_mini.iloc[:,j], dropna=False)
    
        if df_cross.shape[1] == 1:
            df_cross['1'] = 0

        df_cross = df_cross[df_cross.columns[::-1]]
        df_cross = df_cross.transpose()

        if df_cross.shape[1] == 1:
            df_cross.insert(0,'1',0)
            
        a.append(df_cross.to_numpy())
  
    new_df.iloc[:,i] = a


new_df

,CH3 [aliphatic carbon],CH2 [aliphatic carbon],CH [aliphatic carbon],"C [aliphatic carbon No H, not tert]","C [aliphatic carbon No H, not tert].1","C [aliphatic carbon No H, not tert].2",=CH2 [olefinic carbon],=CH or =C [olefinc carbon],#C [acetylenic carbon],"OH [hydroxy, aliphatic attach]",...,"COOH [acid, aliphatic attach]","COOH [acid, aromatic attach]",N=O [nitroso],"S [aliphatic sulfur,one aromatic attach]",Aromatic Sulfur,"C(=O)O [ester, aliphatic attach]","C(=O)O [ester, aromatic attach]","F [fluorine, aromatic attach]",C(=O)N [aliphatic attach],C(=O)N [aromatic attach]
atom:element_main_group,"[[0, 0], [171, 327]]","[[0, 0], [147, 351]]","[[0, 0], [285, 213]]","[[0, 0], [468, 30]]","[[0, 0], [473, 25]]","[[0, 0], [0, 498]]","[[0, 0], [484, 14]]","[[0, 0], [399, 99]]","[[0, 0], [496, 2]]","[[0, 0], [410, 88]]",...,"[[0, 0], [442, 56]]","[[0, 0], [491, 7]]","[[0, 0], [0, 498]]","[[0, 0], [494, 4]]","[[0, 0], [491, 7]]","[[0, 0], [425, 73]]","[[0, 0], [485, 13]]","[[0, 0], [481, 17]]","[[0, 0], [437, 61]]","[[0, 0], [472, 26]]"
atom:element_metal_group_I_II,"[[15, 12], [156, 315]]","[[11, 16], [136, 335]]","[[16, 11], [269, 202]]","[[26, 1], [442, 29]]","[[26, 1], [447, 24]]","[[0, 27], [0, 471]]","[[27, 0], [457, 14]]","[[21, 6], [378, 93]]","[[27, 0], [469, 2]]","[[20, 7], [390, 81]]",...,"[[27, 0], [415, 56]]","[[26, 1], [465, 6]]","[[0, 27], [0, 471]]","[[27, 0], [467, 4]]","[[27, 0], [464, 7]]","[[23, 4], [402, 69]]","[[27, 0], [458, 13]]","[[25, 2], [456, 15]]","[[26, 1], [411, 60]]","[[24, 3], [448, 23]]"
atom:element_metal_group_III,"[[0, 0], [171, 327]]","[[0, 0], [147, 351]]","[[0, 0], [285, 213]]","[[0, 0], [468, 30]]","[[0, 0], [473, 25]]","[[0, 0], [0, 498]]","[[0, 0], [484, 14]]","[[0, 0], [399, 99]]","[[0, 0], [496, 2]]","[[0, 0], [410, 88]]",...,"[[0, 0], [442, 56]]","[[0, 0], [491, 7]]","[[0, 0], [0, 498]]","[[0, 0], [494, 4]]","[[0, 0], [491, 7]]","[[0, 0], [425, 73]]","[[0, 0], [485, 13]]","[[0, 0], [481, 17]]","[[0, 0], [437, 61]]","[[0, 0], [472, 26]]"
atom:element_metal_metalloid,"[[1, 4], [170, 323]]","[[3, 2], [144, 349]]","[[5, 0], [280, 213]]","[[5, 0], [463, 30]]","[[5, 0], [468, 25]]","[[0, 5], [0, 493]]","[[5, 0], [479, 14]]","[[5, 0], [394, 99]]","[[5, 0], [491, 2]]","[[4, 1], [406, 87]]",...,"[[5, 0], [437, 56]]","[[5, 0], [486, 7]]","[[0, 5], [0, 493]]","[[5, 0], [489, 4]]","[[5, 0], [486, 7]]","[[5, 0], [420, 73]]","[[5, 0], [480, 13]]","[[5, 0], [476, 17]]","[[5, 0], [432, 61]]","[[5, 0], [467, 26]]"
atom:element_metal_poor_metal,"[[0, 1], [171, 326]]","[[0, 1], [147, 350]]","[[1, 0], [284, 213]]","[[1, 0], [467, 30]]","[[1, 0], [472, 25]]","[[0, 1], [0, 497]]","[[1, 0], [483, 14]]","[[1, 0], [398, 99]]","[[1, 0], [495, 2]]","[[1, 0], [409, 88]]",...,"[[1, 0], [441, 56]]","[[1, 0], [490, 7]]","[[0, 1], [0, 497]]","[[1, 0], [493, 4]]","[[1, 0], [490, 7]]","[[1, 0], [424, 73]]","[[1, 0], [484, 13]]","[[1, 0], [480, 17]]","[[1, 0], [436, 61]]","[[1, 0], [471, 26]]"
atom:element_metal_transistion_metal,"[[3, 2], [168, 325]]","[[2, 3], [145, 348]]","[[3, 2], [282, 211]]","[[5, 0], [463, 30]]","[[4, 1], [469, 24]]","[[0, 5], [0, 493]]","[[5, 0], [479, 14]]","[[4, 1], [395, 98]]","[[5, 0], [491, 2]]","[[3, 2], [407, 86]]",...,"[[5, 0], [437, 56]]","[[5, 0], [486, 7]]","[[0, 5], [0, 493]]","[[5, 0], [489, 4]]","[[5, 0], [486, 7]]","[[5, 0], [420, 73]]","[[5, 0], [480, 13]]","[[5, 0], [476, 17]]","[[4, 1], [433, 60]]","[[5, 0], [467, 26]]"
atom:element_noble_gas,"[[0, 0], [171, 327]]","[[0, 0], [147, 351]]","[[0, 0], [285, 213]]","[[0, 0], [468, 30]]","[[0, 0], [473, 25]]","[[0, 0], [0, 498]]","[[0, 0], [484, 14]]","[[0, 0], [399, 99]]","[[0, 0], [496, 2]]","[[0, 0], [410, 88]]",...,"[[0, 0], [442, 56]]","[[0, 0], [491, 7]]","[[0, 0], [0, 498]]","[[0, 0], [494, 4]]","[[0, 0], [491, 7]]","[[0, 0], [425, 73]]","[[0, 0], [485, 13]]","[[0, 0], [481, 17]]","[[0, 0], [437, 61]]","[[0, 0], [472, 26]]"
bond:C#N_cyano_acylcyanide,"[[0, 0], [171, 327]]","[[0, 0], [147, 351]]","[[0, 0], [285, 213]]","[[0, 0

In [112]:
new_df.head()

,CH3 [aliphatic carbon],CH2 [aliphatic carbon],CH [aliphatic carbon],"C [aliphatic carbon No H, not tert]","C [aliphatic carbon No H, not tert].1","C [aliphatic carbon No H, not tert].2",=CH2 [olefinic carbon],=CH or =C [olefinc carbon],#C [acetylenic carbon],"OH [hydroxy, aliphatic attach]",...,"COOH [acid, aliphatic attach]","COOH [acid, aromatic attach]",N=O [nitroso],"S [aliphatic sulfur,one aromatic attach]",Aromatic Sulfur,"C(=O)O [ester, aliphatic attach]","C(=O)O [ester, aromatic attach]","F [fluorine, aromatic attach]",C(=O)N [aliphatic attach],C(=O)N [aromatic attach]
atom:element_main_group,"[[0, 0], [171, 327]]","[[0, 0], [147, 351]]","[[0, 0], [285, 213]]","[[0, 0], [468, 30]]","[[0, 0], [473, 25]]","[[0, 0], [0, 498]]","[[0, 0], [484, 14]]","[[0, 0], [399, 99]]","[[0, 0], [496, 2]]","[[0, 0], [410, 88]]",...,"[[0, 0], [442, 56]]","[[0, 0], [491, 7]]","[[0, 0], [0, 498]]","[[0, 0], [494, 4]]","[[0, 0], [491, 7]]","[[0, 0], [425, 73]]","[[0, 0], [485, 13]]","[[0, 0], [481, 17]]","[[0, 0], [437, 61]]","[[0, 0], [472, 26]]"
atom:element_metal_group_I_II,"[[15, 12], [156, 315]]","[[11, 16], [136, 335]]","[[16, 11], [269, 202]]","[[26, 1], [442, 29]]","[[26, 1], [447, 24]]","[[0, 27], [0, 471]]","[[27, 0], [457, 14]]","[[21, 6], [378, 93]]","[[27, 0], [469, 2]]","[[20, 7], [390, 81]]",...,"[[27, 0], [415, 56]]","[[26, 1], [465, 6]]","[[0, 27], [0, 471]]","[[27, 0], [467, 4]]","[[27, 0], [464, 7]]","[[23, 4], [402, 69]]","[[27, 0], [458, 13]]","[[25, 2], [456, 15]]","[[26, 1], [411, 60]]","[[24, 3], [448, 23]]"
atom:element_metal_group_III,"[[0, 0], [171, 327]]","[[0, 0], [147, 351]]","[[0, 0], [285, 213]]","[[0, 0], [468, 30]]","[[0, 0], [473, 25]]","[[0, 0], [0, 498]]","[[0, 0], [484, 14]]","[[0, 0], [399, 99]]","[[0, 0], [496, 2]]","[[0, 0], [410, 88]]",...,"[[0, 0], [442, 56]]","[[0, 0], [491, 7]]","[[0, 0], [0, 498]]","[[0, 0], [494, 4]]","[[0, 0], [491, 7]]","[[0, 0], [425, 73]]","[[0, 0], [485, 13]]","[[0, 0], [481, 17]]","[[0, 0], [437, 61]]","[[0, 0], [472, 26]]"
atom:element_metal_metalloid,"[[1, 4], [170, 323]]","[[3, 2], [144, 349]]","[[5, 0], [280, 213]]","[[5, 0], [463, 30]]","[[5, 0], [468, 25]]","[[0, 5], [0, 493]]","[[5, 0], [479, 14]]","[[5, 0], [394, 99]]","[[5, 0], [491, 2]]","[[4, 1], [406, 87]]",...,"[[5, 0], [437, 56]]","[[5, 0], [486, 7]]","[[0, 5], [0, 493]]","[[5, 0], [489, 4]]","[[5, 0], [486, 7]]","[[5, 0], [420, 73]]","[[5, 0], [480, 13]]","[[5, 0], [476, 17]]","[[5, 0], [432, 61]]","[[5, 0], [467, 26]]"
atom:element_metal_poor_metal,"[[0, 1], [171, 326]]","[[0, 1], [147, 350]]","[[1, 0], [284, 213]]","[[1, 0], [467, 30]]","[[1, 0], [472, 25]]","[[0, 1], [0, 497]]","[[1, 0], [483, 14]]","[[1, 0], [398, 99]]","[[1, 0], [495, 2]]","[[1, 0], [409, 88]]",...,"[[1, 0], [441, 56]]","[[1, 0], [490, 7]]","[[0, 1], [0, 497]]","[[1, 0], [493, 4]]","[[1, 0], [490, 7]]","[[1, 0], [424, 73]]","[[1, 0], [484, 13]]","[[1, 0], [480, 17]]","[[1, 0], [436, 61]]","[[1, 0], [471, 26]]"
